In [1]:
import numpy as np
import pandas as pd
from keras.applications.vgg19 import VGG19
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50

from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
from keras import models

from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2

import sys
from PIL import Image
sys.modules['Image'] = Image 

from PIL import Image
print(Image.__file__)

import os
import shutil

Using TensorFlow backend.


/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/PIL/Image.py


In [16]:
data_dir = 'data'
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
validation_dir = os.path.join(data_dir, 'validation')

df_train = pd.read_csv('data/labels.csv')
df_test = pd.read_csv('data/sample_submission.csv')

im_size = 200

x_train = []
y_train = []
x_test = []


In [3]:

print(df_train.head(10))

targets_series = pd.Series(df_train['breed'])
# print(targets_series)
one_hot = pd.get_dummies(targets_series, sparse=True)

one_hot_labels = np.asarray(one_hot)


                                 id               breed
0  000bec180eb18c7604dcecc8fe0dba07         boston_bull
1  001513dfcb2ffafc82cccf4d8bbaba97               dingo
2  001cdf01b096e06d78e9e5112d419397            pekinese
3  00214f311d5d2247d5dfe4fe24b2303d            bluetick
4  0021f9ceb3235effd7fcde7f7538ed62    golden_retriever
5  002211c81b498ef88e1b40b9abf84e1d  bedlington_terrier
6  00290d3e1fdd27226ba27a8ce248ce85  bedlington_terrier
7  002a283a315af96eaea0e28e7163b21b              borzoi
8  003df8b8a8b05244b1d920bb6cf451f9             basenji
9  0042188c895a2f14ef64a918ed9c7b64  scottish_deerhound


In [9]:
# Create the base pre-trained model
# Can't download weights in the kernel
# base_model = VGG16(
#     weights='imagenet',
#     # weights = None,
#     include_top=False, input_shape=(im_size, im_size, 3))
from keras.layers import merge

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(im_size, im_size, 3), pooling=None)
for layer in base_model.layers:
    layer.trainable = False

input = layers.Input(shape=(im_size, im_size, 3))

# Add the vgg convolutional base model
x = base_model(input)
x = layers.Conv2D(512, (1, 1))(x)

# тут сетка разделяется на две ветки
y = layers.GlobalAveragePooling2D()(x)  # считаем среднее в каждом канале, получаем тензор формы (512,)
z = layers.GlobalMaxPooling2D()(x)  # считаем максимум в каждом канале, получаем тензор формы (512,)
x = layers.concatenate([y, z], axis=-1)  # сливаем вместе эти две ветки, получаем тензор формы (1024,)
res = Dense(120)(x)

# Show a summary of the model. Check the number of trainable parameters

model = Model(inputs=[input], outputs=[res])
print(model.summary())

/home/serg/anaconda3/envs/py36/lib/python3.6/site-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [10]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=2,
      horizontal_flip=True,
      fill_mode='nearest')
 
validation_datagen = ImageDataGenerator(rescale=1./255)
 
# Change the batchsize according to your system RAM
train_batchsize = 16
val_batchsize = 16
 
train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(im_size, im_size),
        batch_size=train_batchsize,
        class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(im_size, im_size),
        batch_size=val_batchsize,
        class_mode='categorical',
        shuffle=False)


Found 7422 images belonging to 120 classes.
Found 2800 images belonging to 120 classes.


In [11]:
# model.fit(X_train, Y_train, epochs=5, validation_data=(X_valid, Y_valid), verbose=1)
# Train the model


model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])


history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)
  


Epoch 1/5


  1/463 [..............................] - ETA: 14:31 - loss: 13.0960 - acc: 0.0625

  2/463 [..............................] - ETA: 14:52 - loss: 10.5775 - acc: 0.0938

KeyboardInterrupt: 

In [12]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)


Epoch 1/5


  1/463 [..............................] - ETA: 14:31 - loss: 13.0960 - acc: 0.0625

  2/463 [..............................] - ETA: 14:52 - loss: 10.5775 - acc: 0.0938

KeyboardInterrupt: 

In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize ,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)

Epoch 1/5


  1/463 [..............................] - ETA: 14:31 - loss: 13.0960 - acc: 0.0625

  2/463 [..............................] - ETA: 14:52 - loss: 10.5775 - acc: 0.0938

KeyboardInterrupt: 

In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=5,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

248/494 [==============>...............] - ETA: 13:24 - loss: 2.0928 - acc: 0.6492

249/494 [==============>...............] - ETA: 13:21 - loss: 2.0933 - acc: 0.6490

250/494 [==============>...............] - ETA: 13:18 - loss: 2.0931 - acc: 0.6485

251/494 [==============>...............] - ETA: 13:15 - loss: 2.0911 - acc: 0.6491

252/494 [==============>...............] - ETA: 13:11 - loss: 2.0887 - acc: 0.6500

In [ ]:

history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=2,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)

model.save('small_last4.h5')


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

248/494 [==============>...............] - ETA: 13:24 - loss: 2.0928 - acc: 0.6492

249/494 [==============>...............] - ETA: 13:21 - loss: 2.0933 - acc: 0.6490

250/494 [==============>...............] - ETA: 13:18 - loss: 2.0931 - acc: 0.6485

251/494 [==============>...............] - ETA: 13:15 - loss: 2.0911 - acc: 0.6491

252/494 [==============>...............] - ETA: 13:11 - loss: 2.0887 - acc: 0.6500

In [23]:


history = model.fit_generator(
      train_generator,
      steps_per_epoch=7422 // train_batchsize,
      epochs=3,
      validation_data=validation_generator,
      validation_steps=2800 // val_batchsize,
      verbose=1)

Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

248/494 [==============>...............] - ETA: 13:24 - loss: 2.0928 - acc: 0.6492

249/494 [==============>...............] - ETA: 13:21 - loss: 2.0933 - acc: 0.6490

250/494 [==============>...............] - ETA: 13:18 - loss: 2.0931 - acc: 0.6485

251/494 [==============>...............] - ETA: 13:15 - loss: 2.0911 - acc: 0.6491

252/494 [==============>...............] - ETA: 13:11 - loss: 2.0887 - acc: 0.6500

In [ ]:

tf = tf.Session(config=tf.ConfigProto(log_device_placement=True))
tf.list_devices()


Epoch 1/3


  1/494 [..............................] - ETA: 45:07 - loss: 0.9508 - acc: 0.6667

  2/494 [..............................] - ETA: 37:12 - loss: 1.5883 - acc: 0.6667

  3/494 [..............................] - ETA: 34:23 - loss: 1.7071 - acc: 0.6889

  4/494 [..............................] - ETA: 32:58 - loss: 1.6005 - acc: 0.7333

...

248/494 [==============>...............] - ETA: 13:24 - loss: 2.0928 - acc: 0.6492

249/494 [==============>...............] - ETA: 13:21 - loss: 2.0933 - acc: 0.6490

250/494 [==============>...............] - ETA: 13:18 - loss: 2.0931 - acc: 0.6485

251/494 [==============>...............] - ETA: 13:15 - loss: 2.0911 - acc: 0.6491

252/494 [==============>...............] - ETA: 13:11 - loss: 2.0887 - acc: 0.6500

In [18]:


x_test = []
for f in tqdm(df_test['id'].values):
    img = cv2.imread('data/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))
    
x_test = np.array(x_test, np.float32) / 255.


  0%|          | 0/10357 [00:00<?, ?it/s]

  1%|          | 64/10357 [00:00<00:16, 638.85it/s]

  1%|          | 123/10357 [00:00<00:16, 622.24it/s]

  2%|▏         | 185/10357 [00:00<00:16, 619.15it/s]

  2%|▏         | 232/10357 [00:00<00:17, 562.88it/s]

...

 98%|█████████▊| 10148/10357 [00:15<00:00, 699.07it/s]

 99%|█████████▊| 10220/10357 [00:15<00:00, 703.68it/s]

 99%|█████████▉| 10291/10357 [00:15<00:00, 674.45it/s]

100%|██████████| 10357/10357 [00:15<00:00, 665.60it/s]

MemoryError: 

In [14]:
preds = model.predict(x_test, verbose=1)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 10357 arrays: [array([[[ 57, 102,  92],
        [ 73, 102, 102],
        [ 84,  99, 107],
        ...,
        [  0, 101,  85],
        [  3, 100,  91],
        [  4,  84,  73]],

       [[ 51, 110, 100],
        [...

In [19]:
sub = pd.DataFrame(preds)

# Set column names to those generated by the one-hot encoding earlier
col_names = one_hot.columns.values
sub.columns = col_names
# Insert the column id from the sample_submission at the start of the data frame
print('started')
sub.insert(0, 'id', df_test['id'])
sub.to_csv('data/submission_resnet_1.csv', encoding='utf-8', index=False)
print('success')

started


success
